### Adjust Repeat Data

In [73]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [74]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [75]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# word usage
min_word_use_num = 1
max_word_use_num = 7

# sentence check
twogram_sentence_check = True  # True, False
threegram_sentence_check = True

# youtube
sample_num = 7  # 7
time_shift = 0.6

In [76]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [77]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column row result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"search_string",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [78]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [79]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")\n
    df_search_result is dataframe and "search_string", "start_time", "end_time", "sentence", "video_id" are its columns
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [80]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [81]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [82]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Result/Adjust Data Repeat"

Path(path).mkdir(parents=True, exist_ok=True)

In [83]:
disable_video_id_list = ["H6E6N70jYqI","BoaYsdPtJYA","HUZINsU40Fk","Et3diPcEmfY"]

In [84]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [85]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [86]:
df_word_select

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
195,genoeg,353128
196,wist,352948
197,oh,351988
198,klaar,350506


In [87]:
word_list = df_word_select["word"].to_list()
len(word_list)

200

In [88]:
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram = df_twogram_sent.loc[:,["twogram","frequency"]]
else:
    df_twogram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")  
    df_twogram = df_twogram.loc[:,["twogram","frequency"]]

df_twogram

,twogram,frequency
0,kom op,184578
1,dank je,127178
2,dank u,64014
3,wacht even,63826
4,echt waar,59238
...,...,...
328541,zesentwintig horizontaal,3
328542,volledig gedisciplineerd,3
328543,volledig geconcentreerd,3
328544,hero netegengif,3


In [89]:
# Twogram Func
d_list2  = df_twogram.iloc[:,0].values.tolist()

resultlist2 = []
manager = multiprocessing.Manager()
resultlist2 = manager.list()

def word_in_wordgroup(d_list2):
    mergelist2 = []
    try:
        word = d_list2.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist2.append(word[j])
            if len(mergelist2) == len(word):
                    resultlist2.append(d_list2)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list2) # string_word liste

result_list2 = list(resultlist2)
df_result2 = pd.DataFrame(result_list2, columns=[0])  # add columns parameter for empty result
df_result2 = df_result2.rename(columns = {0: "twogram"})
df_merge2 = pd.merge(df_result2, df_twogram, how="left", on="twogram")
df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
df_merge_result2.drop_duplicates(inplace=True)
df_merge_result2.reset_index(drop=True, inplace=True)
df_twogram_select = df_merge_result2
df_twogram_select

,twogram,frequency
0,kom op,184578
1,dank je,127178
2,dank u,64014
3,wacht even,63826
4,echt waar,59238
...,...,...
6564,jullie ons,3
6565,dank allemaal,3
6566,nooit weg,3
6567,nooit zeggen,3


In [90]:
df_twogram_word_usage = word_usage_result(word_list, df_twogram_select, "twogram", "frequency", min_word_use_num, max_word_use_num)
df_twogram_word_usage

,twogram,frequency
0,kom op,184578
1,dank je,127178
2,dank u,64014
3,wacht even,63826
4,echt waar,59238
...,...,...
576,toen dacht,3
577,tot tot,3
578,willen weten,3
579,gedaan heb,3


In [91]:
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram = df_threegram_sent.loc[:,["threegram","frequency"]]
else:
    df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")  
    df_threegram = df_threegram.loc[:,["threegram","frequency"]]

df_threegram

,threegram,frequency
0,het spijt me,132758
1,wat is er,106497
2,ik weet het,67518
3,wat is dat,62768
4,wat doe je,60323
...,...,...
550436,vliegen onbekend aantal,3
550437,vliegen obstakel afnemen,3
550438,voor achtduizend euro,3
550439,voor de kijkcijfers,3


In [92]:
# Threegram Func
d_list3  = df_threegram.iloc[:,0].values.tolist()

resultlist3 = []
manager = multiprocessing.Manager()
resultlist3 = manager.list()

def word_in_wordgroup(d_list3):
    mergelist3 = []
    try:
        word = d_list3.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist3.append(word[j])
            if len(mergelist3) == len(word):
                    resultlist3.append(d_list3)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list3) # string_word liste

result_list3 = list(resultlist3)
df_result3 = pd.DataFrame(result_list3, columns=[0])  # add columns parameter for empty result
df_result3 = df_result3.rename(columns = {0: "threegram"})
df_merge3 = pd.merge(df_result3, df_threegram, how="left", on="threegram")
df_merge_result3 = df_merge3.sort_values(by="frequency", ascending=False)
df_merge_result3.drop_duplicates(inplace=True)
df_merge_result3.reset_index(drop=True, inplace=True)
df_threegram_select = df_merge_result3
df_threegram_select

,threegram,frequency
0,het spijt me,132758
1,wat is er,106497
2,ik weet het,67518
3,wat is dat,62768
4,wat doe je,60323
...,...,...
29304,iedereen bij deze,3
29305,misschien had ik,3
29306,nu gaat hij,3
29307,omdat wij weten,3


In [93]:
df_threegram_word_usage = word_usage_result(word_list, df_threegram_select, "threegram", "frequency", min_word_use_num, max_word_use_num)
df_threegram_word_usage

,threegram,frequency
0,het spijt me,132758
1,wat is er,106497
2,ik weet het,67518
3,wat is dat,62768
4,wat doe je,60323
...,...,...
360,echt nodig hebt,3
361,wil gewoon zitten,3
362,zeker weten ben,3
363,dus dus dus,3


In [94]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:10.760,00:00:13.200,jongens hier ben ik dan,WA8MV8nX9s0
1,00:00:13.200,00:00:17.040,en we gaan iets heel maar dan ook heel geks me...,WA8MV8nX9s0
2,00:00:17.040,00:00:19.320,ik bedoel we gaan naar de royal bridge suite,WA8MV8nX9s0
3,00:00:19.900,00:00:23.760,dit is de meest luxe suite die je maar kan bed...,WA8MV8nX9s0
4,00:00:23.760,00:00:25.860,de meest bekende mensen hebben hier geslapen,WA8MV8nX9s0
...,...,...,...,...
517239,00:05:19.400,00:05:21.080,ben ik weer even anders naar buiten gekomen,aHU2ky7LpBk
517240,00:05:21.080,00:05:23.040,zo van hoe ga ik nou met mezelf om,aHU2ky7LpBk
517241,00:05:23.080,00:05:24.160,wat is echt belangrijk,aHU2ky7LpBk
517242,00:05:24.600,00:05:28.480,en dan zie je echt dat de mensheid en liefde d...,aHU2ky7LpBk


In [95]:
df_youtube_sentence = df_youtube_sentence[~df_youtube_sentence["video_id"].isin(disable_video_id_list)]
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:10.760,00:00:13.200,jongens hier ben ik dan,WA8MV8nX9s0
1,00:00:13.200,00:00:17.040,en we gaan iets heel maar dan ook heel geks me...,WA8MV8nX9s0
2,00:00:17.040,00:00:19.320,ik bedoel we gaan naar de royal bridge suite,WA8MV8nX9s0
3,00:00:19.900,00:00:23.760,dit is de meest luxe suite die je maar kan bed...,WA8MV8nX9s0
4,00:00:23.760,00:00:25.860,de meest bekende mensen hebben hier geslapen,WA8MV8nX9s0
...,...,...,...,...
517239,00:05:19.400,00:05:21.080,ben ik weer even anders naar buiten gekomen,aHU2ky7LpBk
517240,00:05:21.080,00:05:23.040,zo van hoe ga ik nou met mezelf om,aHU2ky7LpBk
517241,00:05:23.080,00:05:24.160,wat is echt belangrijk,aHU2ky7LpBk
517242,00:05:24.600,00:05:28.480,en dan zie je echt dat de mensheid en liefde d...,aHU2ky7LpBk


In [96]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

<ipython-input-96-a8911ca4399e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second
<ipython-input-96-a8911ca4399e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])


In [97]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

<ipython-input-97-aa25c83efaa1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
<ipython-input-97-aa25c83efaa1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())


,start_time,end_time,sentence,video_id
0,10.76,13.20,jongens hier ben ik dan,WA8MV8nX9s0
1,13.20,17.04,en we gaan iets heel maar dan ook heel geks me...,WA8MV8nX9s0
2,17.04,19.32,ik bedoel we gaan naar de royal bridge suite,WA8MV8nX9s0
3,19.90,23.76,dit is de meest luxe suite die je maar kan bed...,WA8MV8nX9s0
4,23.76,25.86,de meest bekende mensen hebben hier geslapen,WA8MV8nX9s0
...,...,...,...,...
517239,319.40,321.08,ben ik weer even anders naar buiten gekomen,aHU2ky7LpBk
517240,321.08,323.04,zo van hoe ga ik nou met mezelf om,aHU2ky7LpBk
517241,323.08,324.16,wat is echt belangrijk,aHU2ky7LpBk
517242,324.60,328.48,en dan zie je echt dat de mensheid en liefde d...,aHU2ky7LpBk


In [98]:
#search_list = df_twogram_word_usage["twogram"].to_list()
search_list = df_threegram_word_usage["threegram"].to_list()

In [99]:
len(search_list)

365

In [100]:
df_word_group = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
df_word_group

,search_string,start_time,end_time,sentence,video_id
0,het spijt me,14213.17,14215.320,het spijt me meneer van haga maar we hebben no...,QAjt9liE_a4
1,het spijt me,19147.53,19148.450,het spijt me zeer,QAjt9liE_a4
2,het spijt me,4760.36,4764.360,ik ga niet meer in die stinkdoos het spijt me,UbIX71sXnSY
3,het spijt me,1335.12,1338.614,het spijt me maar de persoon die u belt is nie...,b60UWbsIbDA
4,het spijt me,132.40,135.133,supervervelend het spijt me real sorry,6g6ZsyaJs8Q
...,...,...,...,...,...
1112,dus dus dus,979.77,984.990,dat dus dus dus niet zon goed idee vindt wij z...,WpLsRtnfzuo
1113,dus dus dus,2299.48,2306.890,nee juist ok we komen zijn gekomen zal ja maar...,43CNzte_Alw
1114,dus dus dus,1708.34,1716.050,voor hoeveel geld de feit bij kon drukken mala...,P2rFpOB6ZKI
1115,dus dus dus,1489.46,1493.690,zaden het wordt het niet gepost in je subscrib...,KHud27Cds5U


In [101]:
df_word_group_time_loc = word_group_time_loc(df_word_group, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,het spijt me,14213.170000,14213.484045,het spijt me meneer van haga maar we hebben no...,QAjt9liE_a4
1,het spijt me,19147.530000,19148.233529,het spijt me zeer,QAjt9liE_a4
2,het spijt me,4763.204444,4764.360000,ik ga niet meer in die stinkdoos het spijt me,UbIX71sXnSY
3,het spijt me,1335.120000,1335.889864,het spijt me maar de persoon die u belt is nie...,b60UWbsIbDA
4,het spijt me,133.406895,134.413789,supervervelend het spijt me real sorry,6g6ZsyaJs8Q
...,...,...,...,...,...
1112,dus dus dus,979.944000,980.698000,dat dus dus dus niet zon goed idee vindt wij z...,WpLsRtnfzuo
1113,dus dus dus,2303.507174,2304.554239,nee juist ok we komen zijn gekomen zal ja maar...,43CNzte_Alw
1114,dus dus dus,1712.154421,1713.209474,voor hoeveel geld de feit bij kon drukken mala...,P2rFpOB6ZKI
1115,dus dus dus,1492.176105,1492.754947,zaden het wordt het niet gepost in je subscrib...,KHud27Cds5U


In [102]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,het spijt me,14212.570000,14214.084045,het spijt me meneer van haga maar we hebben no...,QAjt9liE_a4
1,het spijt me,19146.930000,19148.833529,het spijt me zeer,QAjt9liE_a4
2,het spijt me,4762.604444,4764.960000,ik ga niet meer in die stinkdoos het spijt me,UbIX71sXnSY
3,het spijt me,1334.520000,1336.489864,het spijt me maar de persoon die u belt is nie...,b60UWbsIbDA
4,het spijt me,132.806895,135.013789,supervervelend het spijt me real sorry,6g6ZsyaJs8Q
...,...,...,...,...,...
1112,dus dus dus,979.344000,981.298000,dat dus dus dus niet zon goed idee vindt wij z...,WpLsRtnfzuo
1113,dus dus dus,2302.907174,2305.154239,nee juist ok we komen zijn gekomen zal ja maar...,43CNzte_Alw
1114,dus dus dus,1711.554421,1713.809474,voor hoeveel geld de feit bij kon drukken mala...,P2rFpOB6ZKI
1115,dus dus dus,1491.576105,1493.354947,zaden het wordt het niet gepost in je subscrib...,KHud27Cds5U


In [103]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: round(x))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: round(x))
df_word_group_time_loc 

,search_string,start_time,end_time,sentence,video_id
0,het spijt me,14213,14214,het spijt me meneer van haga maar we hebben no...,QAjt9liE_a4
1,het spijt me,19147,19149,het spijt me zeer,QAjt9liE_a4
2,het spijt me,4763,4765,ik ga niet meer in die stinkdoos het spijt me,UbIX71sXnSY
3,het spijt me,1335,1336,het spijt me maar de persoon die u belt is nie...,b60UWbsIbDA
4,het spijt me,133,135,supervervelend het spijt me real sorry,6g6ZsyaJs8Q
...,...,...,...,...,...
1112,dus dus dus,979,981,dat dus dus dus niet zon goed idee vindt wij z...,WpLsRtnfzuo
1113,dus dus dus,2303,2305,nee juist ok we komen zijn gekomen zal ja maar...,43CNzte_Alw
1114,dus dus dus,1712,1714,voor hoeveel geld de feit bij kon drukken mala...,P2rFpOB6ZKI
1115,dus dus dus,1492,1493,zaden het wordt het niet gepost in je subscrib...,KHud27Cds5U


In [104]:
df_word_group_time_loc["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc['video_id'].map(str)+"&t="+df_word_group_time_loc['start_time'].map(str)+"s"
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id,video_url
0,het spijt me,14213,14214,het spijt me meneer van haga maar we hebben no...,QAjt9liE_a4,https://www.youtube.com/watch?v=QAjt9liE_a4&t=...
1,het spijt me,19147,19149,het spijt me zeer,QAjt9liE_a4,https://www.youtube.com/watch?v=QAjt9liE_a4&t=...
2,het spijt me,4763,4765,ik ga niet meer in die stinkdoos het spijt me,UbIX71sXnSY,https://www.youtube.com/watch?v=UbIX71sXnSY&t=...
3,het spijt me,1335,1336,het spijt me maar de persoon die u belt is nie...,b60UWbsIbDA,https://www.youtube.com/watch?v=b60UWbsIbDA&t=...
4,het spijt me,133,135,supervervelend het spijt me real sorry,6g6ZsyaJs8Q,https://www.youtube.com/watch?v=6g6ZsyaJs8Q&t=...
...,...,...,...,...,...,...
1112,dus dus dus,979,981,dat dus dus dus niet zon goed idee vindt wij z...,WpLsRtnfzuo,https://www.youtube.com/watch?v=WpLsRtnfzuo&t=...
1113,dus dus dus,2303,2305,nee juist ok we komen zijn gekomen zal ja maar...,43CNzte_Alw,https://www.youtube.com/watch?v=43CNzte_Alw&t=...
1114,dus dus dus,1712,1714,voor hoeveel geld de feit bij kon drukken mala...,P2rFpOB6ZKI,https://www.youtube.com/watch?v=P2rFpOB6ZKI&t=...
1115,dus dus dus,1492,1493,zaden het wordt het niet gepost in je subscrib...,KHud27Cds5U,https://www.youtube.com/watch?v=KHud27Cds5U&t=...


In [105]:
word_count_result(df_word_group_time_loc, ["search_string"], set_condition=True)

,word,word_count
0,jij,7
1,niet,7
2,wat,7
3,we,7
4,nooit,7
...,...,...
163,zitten,1
164,hou,1
165,komen,1
166,heeft,1


In [ ]:
df_word_group_time_loc.to_excel(f"{lang_folder.capitalize()}_Word_Group_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False) 

#### Repeat Result

In [34]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["search_string","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,search_string,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s


In [35]:
df_result = pd.DataFrame()
for word_group in search_list:
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].head(1)
    df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].iloc[1:,]  # will test
    for word_repeat_num in range(3):
        df_link_default_var = df_link_default
        df_result = pd.concat([df_result,df_word_group_search_repeat1], axis=0)
    for word_repeat_num2 in range(len(df_word_group_search_repeat2)):
        df_word_search_repeat2_var = df_word_group_search_repeat2.iloc[[word_repeat_num2],:]
        try:
            word_time_diff_var = df_word_group_search_repeat2.loc[word_repeat_num2 ,"end_time"] - df_word_group_search_repeat2.loc[word_repeat_num2 ,"start_time"]
            if word_time_diff_var < 4:
                word_time_diff_var2 = 4
            else:
                word_time_diff_var2 = word_time_diff_var+1.0                                       
            df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
            df_result = pd.concat([df_result,df_word_search_repeat2_var], axis=0)
            df_result = pd.concat([df_result,df_link_default_var], axis=0)
        except:
            pass

In [36]:
df_result.reset_index(drop=True, inplace=True)
df_result_join = df_col_value_join_comma(df_result, ["video_id","start_time","end_time"])

In [39]:
df_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time.xlsx", index=False)
df_result_join.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time_Join.xlsx", index=False)

In [37]:
df_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,kom op,241,243,kom op je kan het nee ik kan het niet,rSDPgPDnAng,https://www.youtube.com/watch?v=rSDPgPDnAng&t=...
1,kom op,241,243,kom op je kan het nee ik kan het niet,rSDPgPDnAng,https://www.youtube.com/watch?v=rSDPgPDnAng&t=...
2,kom op,241,243,kom op je kan het nee ik kan het niet,rSDPgPDnAng,https://www.youtube.com/watch?v=rSDPgPDnAng&t=...
3,kom op,381,383,kom op jongens,MyP82uxsHZA,https://www.youtube.com/watch?v=MyP82uxsHZA&t=...
4,repeat,5,9,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s
...,...,...,...,...,...,...
955,willen weten,2228,2231,moeder onderbouwd en ik zou graag willen weten of,6b2CjjmooDA,https://www.youtube.com/watch?v=6b2CjjmooDA&t=...
956,willen weten,2228,2231,moeder onderbouwd en ik zou graag willen weten of,6b2CjjmooDA,https://www.youtube.com/watch?v=6b2CjjmooDA&t=...
957,toen zeker,224,226,en die combinatie hebben wij toen zeker het la...,U4HIBDuiM1Y,https://www.youtube.com/watch?v=U4HIBDuiM1Y&t=...
958,toen zeker,224,226,en die combinatie hebben wij toen zeker het la...,U4HIBDuiM1Y,https://www.youtube.com/watch?v=U4HIBDuiM1Y&t=...


In [38]:
df_result_join

,video_id,start_time,end_time
0,"rSDPgPDnAng,rSDPgPDnAng,rSDPgPDnAng,MyP82uxsHZ...","241,241,241,381,5,1078,5,359,5,181,181,181,458...","243,243,243,383,9,1079,9,361,9,184,184,184,458..."
